In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic/valid.csv
/kaggle/input/arabic/train.csv


# Import Necessary Libraies

In [2]:
!pip install emoji
!pip install arabic-reshaper

In [3]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
import re
import string
import emoji
import random
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loding The Train Data

In [4]:
# reading the train data
df_train =pd.read_csv("/kaggle/input/arabic/train.csv")

In [5]:
# print sample of the data
df_train.head()

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...


In [6]:
df_train.shape

(18350, 2)

# Loding The Valid Data

In [7]:
df_valid=pd.read_csv("/kaggle/input/arabic/valid.csv")

In [8]:
# print a sample of valid data
df_valid.head(10)

,correct,incorrect
0,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...
1,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...
2,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...
3,علي من ميلانو يقول : ان سوريا بغالبيتها التي ت...,علي من ميلانوا يقول ان سوريا بغالبيته التي تبل...
4,قسما بالله ، انا اعرف اهل حمص ، اليوم شيعنا خي...,قسما بالله انا اعرف اهل حمص اليوم شيعنا خيره ا...
5,اصلا لا توجد القاعده في بلاد المغرب العربي ، ب...,اصلا لاتوجد القاعده في بلاد المغرب العربي بل ا...
6,بصراحه ، عندما كانت الجزيره جزيره العرب كنا نش...,بصراحه عندما كانت الجزيره جزيره العرب كنا نشاه...
7,اذا حزب الجنوب اللبناني اعلن الحرب علي الشعب ا...,اذا حزب الجنوب اللبناني اعلن الحرب علي الشعب ا...
8,ان هءلاء القتلي هم شباب قتلهم الخوارج ، ولمن ي...,ان هاءلاء القتلي هم شباب قتلهم الخوارج و لمن ي...
9,بعض الذين يدعمون الارهاب يحاولون تبرءه القتله ...,بعض الذين يدعمون الارهاب يحاولون تبرءه القتله ...


In [9]:
df = pd.concat([df_train, df_valid])
df

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلح جدران بيته ويصبغها وحينما يستعص...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20643 entries, 0 to 2292
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   correct    20643 non-null  object
 1   incorrect  20643 non-null  object
dtypes: object(2)
memory usage: 483.8+ KB


# EDA

In [11]:
stop = set(nltk.corpus.stopwords.words("arabic"))

# Text length analysis
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

# Counting emoji
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

df['word_count_correct'] = df['correct'].apply(lambda x: len(str(x).split(" ")))
df['word_count_incorrect'] = df['incorrect'].apply(lambda x: len(str(x).split(" ")))
df['stopwords_correct'] = df['correct'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['stopwords_incorrect'] = df['incorrect'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['emoji_count_correct'] = df['correct'].apply(lambda x: emoji_counter(x))
df['emoji_count_incorrect'] = df['incorrect'].apply(lambda x: emoji_counter(x))
df.head()

,correct,incorrect,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,65,59,12,8,0,0
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,54,45,10,10,0,0
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,66,57,7,7,0,0
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,70,62,10,10,0,0
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,66,64,11,11,0,0


In [12]:
df.describe()

,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
count,20643.000000,20643.000000,20643.00000,20643.000000,20643.0,20643.0
mean,58.711718,54.240566,9.33561,9.744853,0.0,0.0
std,18.513807,17.561493,4.53619,5.036631,0.0,0.0
min,6.000000,6.000000,0.00000,0.000000,0.0,0.0
25%,51.000000,47.000000,7.00000,7.000000,0.0,0.0
50%,57.000000,52.000000,9.00000,9.000000,0.0,0.0
75%,63.000000,58.000000,11.00000,12.000000,0.0,0.0
max,484.000000,461.000000,132.00000,132.000000,0.0,0.0


# Text Preprossesing

# Meak a incorrect data

In [13]:
# Function to add a random character to a word in Arabic text
def add_random_arabic_character(word):
    arabic_chars = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"
    random_char = random.choice(arabic_chars)
    return word + random_char

# Function to remove a random character from a word in Arabic text
def remove_random_arabic_character(word):
    if len(word) <= 1:
        return word  # If the word has only one character, return the same word
    random_index = random.randint(0, len(word) - 1)  # Choose a random index
    return word[:random_index] + word[random_index + 1:]  # Remove the character at the random index


In [14]:
def modify_text(text):
    words = text.split()  # Split the text into words
    if not words:
        return ""  # Return empty string if there are no words
    random_word = random.choice(words)  # Choose a random word
    modify_choice = random.choice(["add", "remove"])  # Choose whether to add or remove a character
    modified_words = []
    for word in words:
        if word == random_word:
            if modify_choice == "add":
                modified_words.append(add_random_arabic_character(word))
            elif modify_choice == "remove":
                modified_words.append(remove_random_arabic_character(word))
        else:
            modified_words.append(word)
    return ' '.join(modified_words)  # Join the modified words back into a single string


In [15]:
df['new_incorrect'] = df['correct'].apply(modify_text)

In [16]:
df.head(10)

,correct,incorrect,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect,new_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,65,59,12,8,0,0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,54,45,10,10,0,0,النصر ات لا محال ان شاء الله . من يءمن بالله و...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,66,57,7,7,0,0,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,70,62,10,10,0,0,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,66,64,11,11,0,0,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
5,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...,سوريا اما للسوريين او لبشار وعصابته لن تنتهي ه...,62,52,8,8,0,0,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...
6,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون و ...,50,49,4,9,0,0,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...
7,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا نحن...,53,52,10,14,0,0,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...
8,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...,تنويه : عقرب هي قريه تقع في سهل الحوله وتتبع ا...,49,45,7,9,0,0,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...
9,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...,ضحكو علينا بحرب العراق عام ١٩٩٠ وعام ٢٠٠٣ وحرب...,56,50,1,2,0,0,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...


In [17]:
df.iloc[3][1]

'الاسد وعصابته لحد الان غير مستوعبين ان و رح ينقلعوا من فوق رقاب الشعب السوري . . . يعني لس ع راكبه براسون انو هالشعب قطيع غنم مابيحل ولا بيربط ونهايتو الدبح ! تماما كالقذافي ومن اثر الصدمه بدات التصر فات العنيفه بالقتل والاعتقال والتعذيب . . اعتقد بان الكثيرين سوف لن يفيقوا من الصدمه ابدا لانهم سيلحقوا بشهاءنا باذن الرب .'

# now we will select the column that we need for model

In [18]:
df=df[["correct","new_incorrect"]]
print(df.iloc[5][0],' \n\n',df.iloc[5][-1])

سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنتهي هذه الثوره الا بان تبقي سوريا حره ، ليس فيها مكان لقتله الاطفال والنساء ، الشعب السوري لن يرضي ان يحكمه خاءن قتل ابناء سوريا . لو استمرت الثوره للاجيال القادمه ، لن يخذل الشهداء ، وسوف تنتصر الثوره ، وستجر اذيال الخيبه عصابه الطغيان . الله معكم يا احرار سوريا .  

 سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنتهي هذه الثوره الا بان تبقي سوريا حره ، ليس فيها مكان لقتله الاطفال والنساء ، الشعب السوري لن يرضي ان يحكمه خاءن قتل ابناء سوريا . لو استمرت الثوره للاجيال القادمه ، لن يخذل الشهداء ، وسوف تنتصر الثوره ، وستجر اذيال الخيبه عصابه الطغيان . الل معكم يا احرار سوريا .


# Removing Punctuations

In [19]:
arabicPunctuations = [".","#","$","//","?","=","'","_","-","'",";","\\","`","؛","<",">","(",")","*","&","^","%","]","[",",","ـ","،","/",":","؟",".","'","{","}","~","|","!","”","…","“","–"] # defining customized punctuation marks
englishPunctuations = string.punctuation # importing English punctuation marks
englishPunctuations = [word.strip() for word in englishPunctuations] # converting the English punctuation from a string to array for processing
punctuationsList = arabicPunctuations + englishPunctuations # creating a list of all punctuation marks

def removingPunctuation(text):
  cleanTweet = ''
  for i in text:
    if i not in arabicPunctuations:
      cleanTweet = cleanTweet + '' + i
  return cleanTweet

df['correct'] = df['correct'].apply(lambda x: removingPunctuation(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x: removingPunctuation(x))


df.sample(5)

,correct,new_incorrect
4850,محمد علي عقيل وناءل الدخيل هم من مءيدين الرءيس...,محمد علي عقيل وناءل الدخيل هم من مءيدين الرءيس...
1331,جاء رسول قيصر الي سيدنا عمر بن الخطاب فوجده ن...,جاء رسول قيصر الي سيدنا عمر بن الخطاب فوجده ن...
1565,اللهم اغفر جميع من استشهد في هذه العمليه الجبا...,اللهم اغفر جميع من استشهد في هذه العمليه الجبا...
17159,لقد تعبنا من الدراسات التي تاره تنفي وتاره توك...,لقد تعبنا من الدراسات اتي تاره تنفي وتاره توكد...
6287,لا لدستور ملكي نعم لدستور من الشعب الي الشعب ...,ا لدستور ملكي نعم لدستور من الشعب الي الشعب ...


In [20]:
def clean(text):
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    # Remove numbers
    text = re.sub("\d+", " ", text)
    return text

df['correct'] = df['correct'].apply(lambda x:clean(x))
df['correct'] = df['correct'].apply(lambda x:clean(x))

df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))

In [21]:
df

,correct,new_incorrect
0,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...
1,النصر ات لا محال ان شاء الله من يءمن بالله وال...,النصر ات لا محال ان شاء الله من يءمن بالله وال...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,وهو يصلح جدران بيته ويصبغها وحينما يستعصي علي...,وهو يصلح جدران بيته ويصبغها وحينما يستعصي علي...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [22]:
df.sample(10)

,correct,new_incorrect
14549,انا لست مع الحكومه المصريه في كثير مما تفعل لك...,انا لست مع الحكومه المصريه في كثير مما تفعل كن...
2109,المشكله اين الملايين تتفرج علي الكره ام علي مر...,المشكله اين الملايين تتفرج علي الكره ام علي مر...
10688,"قال الله تعالي "" وما كيد الظالمين الا في ضلال ...","قال اله تعالي "" وما كيد الظالمين الا في ضلال ""..."
3408,الي الاضراب الي الاضراب يا احفاد العرب الامجاد...,الي الاضراب الي الاضراب يا احفاد العرب الامجاد...
1170,يا شعوب العالم عليكم بالاسلام يا شباب العالم ع...,يا شعوب العالم عليكم بالاسلام يا شباب العالم ع...
15451,طالما بشار ونظامه من منعوا التحقيق فهو بالتاكي...,طالما بشار ونظامه من منعوا التحقيق فهو بالتاكي...
798,شكرا للجزيره منبر من لا منبر له علي تغطيتها ال...,شكرا للجزيره منبر من لا منبر له علي تغطيتها ال...
18284,ايها الجهال يا ايها العملاء اتقتلون النفس التي...,ايها الجهال يا ايها العملاء اتقتلون النفس التي...
13220,ان الاسلام المرضي من امريكا هو الاسلام الشيعي ...,ان الاسلام المرضي من امريكا هو الاسلام الشيعي ...
18106,تجريب المجرب خطا مرتين اعتقد هذا تصرف غبي من ن...,تجريب المجرب خطا مرتين اعتقد هذا تصرف غبي من ن...


# Tokenize the text data

In [23]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(df['correct'])

# Convert text data to sequences of integers

In [24]:
correct_sequences = tokenizer.texts_to_sequences(df['correct'])
incorrect_sequences = tokenizer.texts_to_sequences(df['new_incorrect'])

In [25]:
# Set the maximum sequence length
max_sequence_length = max(max(len(seq) for seq in correct_sequences), max(len(seq) for seq in incorrect_sequences))

# Pad sequences to ensure uniform length
correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')

In [26]:
len(tokenizer.word_index)

100874

In [27]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(incorrect_sequences, correct_sequences, test_size=0.2, random_state=42)

# Model

In [28]:
# Model architecture
input_seq = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_seq)
encoder = LSTM(128, return_sequences=True)(embedding_layer)
decoder = Dense(len(tokenizer.word_index) + 1, activation='softmax')(encoder)

# Define the model
model = Model(input_seq, decoder)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 394)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 394, 128)            │      12,912,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 394, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 394, 100875)         │      13,012,875 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,056,459 (99.40 MB)

 Trainable params: 26,056,459 (99.40 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=16)

Epoch 1/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 735s 708ms/step - accuracy: 0.8675 - loss: 2.4295 - val_accuracy: 0.8764 - val_loss: 1.1044
Epoch 2/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 746s 722ms/step - accuracy: 0.8793 - loss: 1.0732 - val_accuracy: 0.8908 - val_loss: 1.0139
Epoch 3/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 746s 723ms/step - accuracy: 0.8953 - loss: 0.9435 - val_accuracy: 0.9062 - val_loss: 0.8890
Epoch 4/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 747s 723ms/step - accuracy: 0.9098 - loss: 0.7946 - val_accuracy: 0.9166 - val_loss: 0.7903
Epoch 5/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 749s 725ms/step - accuracy: 0.9235 - loss: 0.6481 - val_accuracy: 0.9227 - val_loss: 0.7285
Epoch 6/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 752s 728ms/step - accuracy: 0.9349 - loss: 0.5334 - val_accuracy: 0.9262 - val_loss: 0.6813
Epoch 7/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 753s 729ms/step - accuracy: 0.9443 - loss: 0.4409 - val_accuracy: 0.9292 - val_loss: 0.6522
Epoch 8/16
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 756s 732ms/step - ac

# Save The Model

In [31]:
# Save the model
model.save('auto_correction_model.h5')

In [32]:
# Tokenize the input text
input_text =str(input())
input_sequence = tokenizer.texts_to_sequences([input_text])

# Pad the input sequence
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

# Predict the corrected output
output_sequence = model.predict(padded_input_sequence)
corrected_output_sequence = np.argmax(output_sequence, axis=-1)

# Convert the corrected output sequence back to text
corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

print("Corrected Output:", corrected_output_text)

 علي العب العربي التركيز


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Corrected Output: ['علي الشعب العربي التركيز']


In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
# Assuming you already have a tokenizer object named 'tokenizer'

# Save the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

In [34]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Result

In [35]:
def predict(input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])

    # Pad the input sequence
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

    # Predict the corrected output
    output_sequence = model.predict(padded_input_sequence)
    corrected_output_sequence = np.argmax(output_sequence, axis=-1)

    # Convert the corrected output sequence back to text
    corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

    return corrected_output_text

In [36]:
sentences = [
    "علي العب العربي التركيز",
    "يجب قول الحف",
    "الحياه ف سوريا صعبه",
    "سلام علي الوطن العبي",
    "اذا اردت ان تضحك ليك بمشاهده برامج التلفزيون",
    "الذهاب و العوده انر صعب",
    "اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء",
]
len (sentences)

7

In [37]:
prediction=[]
for i in range(len(sentences)):
    corrected_text = predict(sentences[i])
    print("Corrected text:", corrected_text)
    prediction.append(corrected_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['علي الشعب العربي التركيز']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['يجب قول']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['الحياه في سوريا صعبه']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['سلام علي الوطن تعدي']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['اذا اردت ان تضحك بمشاهده برامج التلفزيون']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['الذهاب و العوده صعب']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Corrected text: ['اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء']
